# Libraries Import

In [1]:
# LIBRARIES IMPORT

# for working with dataframes, CSV
import pandas as pd

# for working with twitter API
import tweepy

# for working with wrapping python codes into SQL queries
import sqlalchemy as db

# for getting current date and time for CSV file creation
from datetime import datetime

# for keeping credentials out of sight
import os
from dotenv import load_dotenv

# for sentiment analysis
from textblob import TextBlob

# Credentials and Authorization

In [2]:
# establish working directory path
# getcwd() returns current working directory
wdir_path = os.getcwd()

### Twitter Credentials

In [3]:
# create a "twitter-keys.env" file under the script working directory
# fill in personal api keys and access tokens and save

twitter_path = os.path.join(wdir_path, "twitter-keys.env") # absolute path of "twitter-keys.env"
# load the credentials into os environment 
load_dotenv(twitter_path)
#check if credentials loaded successfully
os.environ


# Credentials obtained from twitter developer account to access API
# getting api_keys & access_tokens from "twitter-keys.env"
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
access_key = os.getenv("access_key")
access_secret = os.getenv("access_secret")

# setup Twitter API connection details
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

### SQL Credentials

In [4]:
# create a "sql-keys.env" file under the script working directory
# fill in personal api keys and access tokens and save
sql_path = os.path.join(wdir_path, "sql-keys.env") # absolute path of "sql-keys.env"
# load the credentials into os environment 
load_dotenv(sql_path)
#check if credentials loaded successfully
os.environ

# getting credentials information from "sql-keys.env"
user = os.getenv("dbUser")
password = os.getenv("dbPwd")
hostname = os.getenv("dbHost")
port_no = os.getenv("dbPort")
db_name = os.getenv("dbName")

# setup connection engine details
engine = db.create_engine(f"postgresql://{user}:{password}@{hostname}:{port_no}/{db_name}") 

# Create Table in PostgreSQL

In [5]:
# Create connection engine
conn = engine.raw_connection()

# Create new tables in PostgreSQL
commands = ("""CREATE TABLE IF NOT EXISTS twitterDatabase (tweet_id BIGINT PRIMARY KEY,
                                                           username VARCHAR(15),
                                                           display_name TEXT,
                                                           location TEXT,
                                                           followers_count INTEGER,
                                                           following_count INTEGER,
                                                           tweet_text TEXT,
                                                           hashtags TEXT,
                                                           polarity NUMERIC(3,2),
                                                           subjectivity NUMERIC(3,2));""")

# Initialize connection to PostgreSQL
cur = conn.cursor()
table_count = 0

# Create cursor to execute SQL commands
cur.execute(commands)
table_count += 1

# Close communication with server
conn.commit()
cur.close()
conn.close()

print(str(table_count),"table(s) have been created in PostgreSQL.")

1 table(s) have been created in PostgreSQL.


# Data Extraction

Creation of queries using Tweepy API

Function is focused on completing the query then providing a CSV file of that query using pandas

And then inserting the data directly into PostgreSQL database

In [6]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
        print()
        print(f"====Tweet {n}====")
        print(f"Tweet ID:{ith_tweet[0]}")
        print(f"Userame:{ith_tweet[1]}")
        print(f"Display Name:{ith_tweet[2]}")
        print(f"Location:{ith_tweet[3]}")
        print(f"Follower Count:{ith_tweet[4]}")
        print(f"Following Count:{ith_tweet[5]}")
        print(f"Tweet Text:{ith_tweet[6]}")
        print(f"Hashtags Used:{ith_tweet[7]}")
        print(f"Polarity:{ith_tweet[8]}")
        print(f"Subjectivity:{ith_tweet[9]}")

# function to perform data extraction
def scrape(words, numtweet):

        # Creating DataFrame using pandas
        tweetsDF = pd.DataFrame(columns=["tweet_id",
                                   "username",
                                   "display_name",
                                   "location",
                                   "followers",
                                   "following",
                                   "text",
                                   "hashtags",
                                   'polarity',
                                   'subjectivity'])

        # We are using .Cursor() to search through twitter for the required tweets.
        # The number of tweets can be restricted using .items(number of tweets)
        if inp == "1": # if user input to search using hashtag
            tweets = tweepy.Cursor(api.search_tweets,
                                   words,
                                   lang="en",
                                   tweet_mode="extended").items(numtweet)
        elif inp == "2": # if user input to search using username
            tweets = tweepy.Cursor(api.user_timeline,
                                   screen_name=words,
                                   tweet_mode="extended").items(numtweet)
                
        
        # .Cursor() returns an iterable object.
        # Each item in the iterator has various attributes
        # that you can access to get information about each tweet
        list_tweets = [tweet for tweet in tweets]
        
        # we will iterate over each tweet in the list for extracting information about each tweet
        val = []
        for i, tweet in enumerate(list_tweets, 1):
            tweet_id = tweet.id
            username = tweet.user.screen_name
            display_name = tweet.user.name
            location = tweet.user.location
            followers = tweet.user.followers_count
            following = tweet.user.friends_count
            hashtags = tweet.entities["hashtags"]
            
            # Retweets can be distinguished by a retweeted_status attribute,
            # in case it is an invalid reference, except block will be executed
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:
                text = tweet.full_text
            
            # Sentiment analysis
            polarity = round(TextBlob(text).sentiment[0], 2)
            subjectivity = round(TextBlob(text).sentiment[1], 2)            
            
            # extracting all hashtags in the tweet
            # because there may be multiple hashtags in a tweet
            hashtext = list()            
            for j in range(0, len(hashtags)):
                hashtext.append(hashtags[j]["text"])
                
            # Here we are appending all the extracted information in the DataFrame
            ith_tweet = [tweet_id, username,
                         display_name, location,
                         followers, following,
                         text, hashtext,
                         polarity, subjectivity]
            tweetsDF.loc[len(tweetsDF)] = ith_tweet

            # Function call to print tweet data on screen
            printtweetdata(i, ith_tweet)
            
            val.append(ith_tweet)
            

        # we will save our database as a CSV file.
        # depending on user's input to search by hashtag or username
        # CSV filename generated will be different
        if inp == "1": # if user input to search using hashtag
            tweetsDF.to_csv("hashtag-{}-tweets_{}.csv".format(words, datetime.now().strftime("%Y-%m-%d_%H%M%S")), sep=",", index=False)
        elif inp == "2": # if user input to search using username
            tweetsDF.to_csv('{}-tweets_{}.csv'.format(username, datetime.now().strftime("%Y-%m-%d_%H%M%S")), sep=',', index=False)
        

        # SQL command to insert scrapped tweet data into PostgreSQL database
        sql = """
        INSERT INTO twitterDatabase(tweet_id, username, display_name, location, followers_count, following_count,
        tweet_text, hashtags, polarity, subjectivity)
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
        ) ON CONFLICT(tweet_id) DO NOTHING
        """
        
        # Create connection engine
        conn = engine.raw_connection()
        
        # Initialize connection to PostgreSQL
        cur = conn.cursor()
        
        # Create cursor to execute SQL commands
        cur.executemany(sql, val)
        
        # print message when scrape is successful
        print("\nScraping has completed!")

        # Close communication with server
        conn.commit()
        cur.close()
        conn.close()      
        

# User Search Query

Querying user if they want to search by hashtag or username

Then querying for user's input based on their choice

In [8]:
# function for user search query
def search_input():

    # To make sure user enters only valid inputs; must be alphanumeric.
    while True:
        try:
            words = input()
            # If input is empty
            if not words:
                raise ValueError("You did not enter anything. Please try again.")
            # If input does not contain only alphanumeric.
            elif not words.isalnum():
                raise ValueError("Only alphanumeric allowed. Please try again.")
            break
        except ValueError as e: # to catch any other errors.
            print(e)

    numtweet = 100

    print("\nLoading, please wait......")
    
    scrape(words, numtweet)
    
# function to query if user wants to search by hashtag or username
def one_or_two():
    # asking user how they want to search
    print("Do you want to:\n1) Search by a hashtag, or\n2) Search by a username?")
    global inp # declaring this inp as global so it can be used to check user's input in other functions
    inp = input()
    if inp == "1": # if user choose to search by hashtag
        print("Enter Twitter HashTag to search for.")
        search_input()
    elif inp == "2": # if user to choose to search by username
        print("Enter Twitter Username to search for.")
        search_input()
    else: # if user did not input 1 or 2
        print("You must choose between 1 or 2\n")
        return one_or_two()
one_or_two()

Do you want to:
1) Search by a hashtag, or
2) Search by a username?
1
Enter Twitter HashTag to search for.
StandWithUkraine

Loading, please wait......

====Tweet 1====
Tweet ID:1507426577029677061
Userame:expatua
Display Name:Expat in Kyiv
Location:Kyiv, Ukraine
Follower Count:7825
Following Count:1089
Tweet Text:Love is watching together a Russian warship burning.
Source spokesman of the Odesa Military Administration.
#StandWithUkraine https://t.co/aPUYsKAMzC
Hashtags Used:['StandWithUkraine']
Polarity:0.13
Subjectivity:0.23

====Tweet 2====
Tweet ID:1507426572004864000
Userame:lolagomezucla
Display Name:Lola
Location:California, USA
Follower Count:164
Following Count:427
Tweet Text:Volodymyr Klitschko: Month of shame. #Ukraine is not a battlefield. It is a crime scene.

#StopRussia #StandWithUkraine https://t.co/chid2EZXnx
Hashtags Used:['Ukraine', 'StopRussia', 'StandWithUkraine']
Polarity:0.0
Subjectivity:0.0

====Tweet 3====
Tweet ID:1507426567399555119
Userame:Harvard2H
Display 


====Tweet 28====
Tweet ID:1507426405189144579
Userame:malcolm291956
Display Name:malcolm taylor
Location:
Follower Count:1962
Following Count:2048
Tweet Text:Ukrainian women are in the forefront of the battle for our independence and freedom. 
#StandWithUkraine https://t.co/BDEw4vupHO
Hashtags Used:['StandWithUkraine']
Polarity:0.0
Subjectivity:0.0

====Tweet 29====
Tweet ID:1507426396745908249
Userame:lwoodard1218
Display Name:L-Train
Location:United States
Follower Count:219
Following Count:459
Tweet Text:BREAKING: The heroic Snake Island sailors who went viral for shouting "Russian warship, go fuck yourself" were freed on Thursday during a prisoner swap with Russia.
#StandWithUkraine #SlavaUkraini 🇺🇦
Hashtags Used:[]
Polarity:0.1
Subjectivity:0.5

====Tweet 30====
Tweet ID:1507426388382474244
Userame:Auriandra
Display Name:🇺🇦 Auriandra 🌻 Слава Україні!
Location:Lake Wobegon MN
Follower Count:4279
Following Count:4158
Tweet Text:Warsaw! @POTUS  #StandWithUkraine https://t.co/4fkzHty


====Tweet 67====
Tweet ID:1507426158253649922
Userame:Ian_in_Formby
Display Name:Ian Bryden
Location:
Follower Count:172
Following Count:72
Tweet Text:📣"Come to your squares, make yourself visible, show your support to the Ukrainian people".

President Zelensky has made the call.

Tomorrow London will respond. 

📅 Saturday at 2pm
📌Park Lane to Trafalgar Square

⬇️ Join us to #StandWithUkraine 💙💛
https://t.co/cXcomGr4zw https://t.co/SJ8t9UcqFd
Hashtags Used:[]
Polarity:0.0
Subjectivity:0.0

====Tweet 68====
Tweet ID:1507426156122832896
Userame:kmeeo
Display Name:kim o Pro-Democracy!
Location:Colorado
Follower Count:16064
Following Count:16192
Tweet Text:Poor guy. Bit off more than he could chew.

He thought he could waltz right into Ukraine, and they would greet them with open arms!

Little did he know, was how independent Ukraine wants to stay.

                 #StandWithUkraine

#wtpBLUE #ONEV1 #DemVoice1 https://t.co/3NaxgrOORD
Hashtags Used:[]
Polarity:0.06
Subjectivity:0.41

====

====Tweet 86====
Tweet ID:1507425977017806859
Userame:3Patriot8
Display Name:Cozypuppy
Location:
Follower Count:61
Following Count:119
Tweet Text:Ex-supreme allied commander of NATO, General Breedlove: ‘Putin is extremely unhappy with the performance of his military right now’
#StandWithUkraine
Hashtags Used:[]
Polarity:-0.09
Subjectivity:0.51

====Tweet 87====
Tweet ID:1507425971661684736
Userame:MacSond
Display Name:Maciej D Sondÿ ♣
Location:WarSaw
Follower Count:1341
Following Count:300
Tweet Text:Warsaw! @POTUS  #StandWithUkraine https://t.co/4fkzHty52r
Hashtags Used:['StandWithUkraine']
Polarity:0.0
Subjectivity:0.0

====Tweet 88====
Tweet ID:1507425953076625435
Userame:pjakobs
Display Name:Peter Jakobs 🤔 (⛵️)
Location:///beakers.sooner.potently
Follower Count:1909
Following Count:2276
Tweet Text:Warsaw! @POTUS  #StandWithUkraine https://t.co/4fkzHty52r
Hashtags Used:['StandWithUkraine']
Polarity:0.0
Subjectivity:0.0

====Tweet 89====
Tweet ID:1507425951449235487
Userame:BanuRodi
